In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors as mcolors
import random

import geopandas as gpd
import numpy as np
from shapely.geometry import Point,Polygon

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

from ipywidgets import widgets
from ipywidgets import interact, interact_manual


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


### Data exploration of census data

In [5]:
census=pd.read_csv('/Users/Vineeth/Google Drive/Insight_project/neighbourhoods/neighbourhood-profiles-2016-csv.csv')
census.shape

(2383, 146)

In [6]:
print(census.columns)

Index(['_id', 'Category', 'Topic', 'Data Source', 'Characteristic',
       'City of Toronto', 'Agincourt North', 'Agincourt South-Malvern West',
       'Alderwood', 'Annex',
       ...
       'Willowdale West', 'Willowridge-Martingrove-Richview', 'Woburn',
       'Woodbine Corridor', 'Woodbine-Lumsden', 'Wychwood', 'Yonge-Eglinton',
       'Yonge-St.Clair', 'York University Heights', 'Yorkdale-Glen Park'],
      dtype='object', length=146)


In [7]:
census.head(2)

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No 

**Explore the different values in the category column !**

In [8]:
census['Category'].value_counts()

Language                                   808
Income                                     311
Language of work                           280
Ethnic origin                              279
Education                                  182
Immigration and citizenship                154
Labour                                      76
Housing                                     74
Families, households and marital status     70
Population                                  55
Aboriginal peoples                          34
Journey to work                             25
Mobility                                    18
Visible minority                            15
Neighbourhood Information                    2
Name: Category, dtype: int64

**Explore the different values in the Topics column !**

In [9]:
census['Topic'].value_counts()

Ethnic origin population                                                                            279
Language used most often at work                                                                    269
Language spoken most often at home                                                                  269
Knowledge of languages                                                                              263
Mother tongue                                                                                       253
Major field of study - Classification of Instructional Programs (CIP) 2016                          124
Income sources                                                                                       94
Income of individuals in 2015                                                                        80
Recent immigrants by selected place of birth                                                         61
Immigrants by selected place of birth                           

In [10]:
len(census['Characteristic'].unique())

1651

In [11]:
census['Characteristic'].unique()[0:10]

array(['Neighbourhood Number', 'TSNS2020 Designation', 'Population, 2016',
       'Population, 2011', 'Population Change 2011-2016',
       'Total private dwellings',
       'Private dwellings occupied by usual residents',
       'Population density per square kilometre',
       'Land area in square kilometres', 'Children (0-14 years)'],
      dtype=object)

## Selection of features in the 'Characteristic' column

Features traditionally important for customer segementation are selected manually to avoid redudancy. The choosen categories are 

1. Population
2. Income
3. marital status
4. household type
5. education
6. ethinicity
7. immigrant status

In [39]:
population_index=[4,7,17,31,18,39,19,40,20,41,21,42,22,43,23,44,24,45,
            25,46,26,47,27,48,28,49,29,50,30,51,32,52,33,53,34,54,35,55]
income_index=[1055,1056,1057,1058,1059,1060,1061,1062,1063,
             1064,1065,1066,1067,1068,1069,1071,1072,1073]
marital_index=[76,80,81,82,83]
household_index=[58,59,61,62,63,64,68,69,70,71,72]
education_index=[1715,1716,1718,1722,1723,1725,1726,1727,1720,1728]
ethnic_index=[1350,1354,1366,1374,1379,1390,1397,1414,1435,1399,1458,1483,
                   1551,1582,1598,1619]
immigrant_index=[1151,1153,1154,1156,1157,1160,1161]

### confirm the selected categories

In [40]:
print(census['Characteristic'].loc[population_index+income_index+marital_index+household_index+education_index+ethnic_index+immigrant_index].tolist())

['Population Change 2011-2016', 'Population density per square kilometre', 'Male: 10 to 14 years', 'Female: 10 to 14 years', 'Male: 15 to 19 years', 'Female: 15 to 19 years', 'Male: 20 to 24 years', 'Female: 20 to 24 years', 'Male: 25 to 29 years', 'Female: 25 to 29 years', 'Male: 30 to 34 years', 'Female: 30 to 34 years', 'Male: 35 to 39 years', 'Female: 35 to 39 years', 'Male: 40 to 44 years', 'Female: 40 to 44 years', 'Male: 45 to 49 years', 'Female: 45 to 49 years', 'Male: 50 to 54 years', 'Female: 50 to 54 years', 'Male: 55 to 59 years', 'Female: 55 to 59 years', 'Male: 60 to 64 years', 'Female: 60 to 64 years', 'Male: 65 to 69 years', 'Female: 65 to 69 years', 'Male: 70 to 74 years', 'Female: 70 to 74 years', 'Male: 75 to 79 years', 'Female: 75 to 79 years', 'Male: 80 to 84 years', 'Female: 80 to 84 years', 'Male: 85 to 89 years', 'Female: 85 to 89 years', 'Male: 90 to 94 years', 'Female: 90 to 94 years', 'Male: 95 to 99 years', 'Female: 95 to 99 years', '  Under $5,000', '  $5,0

### Filter the data based on the selected features

In [41]:
census_sel=census.loc[population_index+education_index+marital_index+household_index+income_index+
          immigrant_index+ethnic_index].iloc[:,4:]
census_sel.index=census_sel['Characteristic']
census_sel.drop(labels=['Characteristic','City of Toronto'],axis=1,inplace=True)
census_sel.head()

,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
Characteristic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Population Change 2011-2016,-3.90%,8.00%,1.30%,4.60%,2.90%,2.80%,33.30%,21.10%,-2.80%,0.20%,1.40%,7.50%,2.00%,-1.50%,-0.50%,-0.30%,6.30%,-0.60%,-0.20%,-3.20%,1.10%,4.60%,2.10%,10.60%,8.90%,12.70%,1.50%,2.80%,2.40%,2.80%,1.20%,2.60%,5.80%,1.10%,2.90%,2.60%,4.00%,-0.20%,-1.00%,1.30%,-1.20%,8.40%,-1.10%,2.70%,-1.80%,-2.90%,2.40%,1.00%,38.70%,4.10%,10.10%,-4.60%,-4.10%,3.40%,-0.90%,-1.90%,1.80%,4.20%,15.40%,2.40%,3.90%,0.40%,-3.00%,1.30%,1.10%,0.80%,-2.10%,10.40%,0.50%,0.70%,-1.10%,29.10%,4.70%,-2.90%,-0.80%,1.10%,-2.20%,28.00%,-0.80%,25.80%,3.40%,0.50%,5.00%,3.70%,5.20%,-2.00%,3.40%,46.60%,-2.30%,4.40%,2.60%,0.70%,2.00%,1.30%,0.60%,0.50%,23.10%,2.00%,-2.00%,-1.30%,8.00%,0.40%,-0.10%,-0.50%,1.40%,1.30%,4.50%,-0.10%,0.70%,2.80%,8.70%,-0.80%,-1.60%,1.50%,0.20%,0.60%,2.10%,2.20%,9.80%,-1.50%,-2.20%,1.90%,52.00%,3.20%,-2.30%,3.30%,-1.00%,-7.60%,3.30%,12.00%,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%
Population density per square kilometre,"3,929","3,034","2,435","10,863","2,775","3,377","14,097","4,195","3,240","4,209","3,614","4,011","3,765","6,282","8,134","7,791","1,040","6,764","5,045","8,335","6,464","5,683","2,479","23,044","3,632","3,979","2,273","7,478","8,554","7,881","6,441","4,146","9,819","2,337","8,540","8,038","2,840","7,051","3,306","6,466","2,171","6,582","9,026","8,054","4,380","5,864","8,582","2,673","6,047","11,726","4,893","

In [42]:
census_sel.shape

(105, 140)

### Invert the dataframe to make neighbourhoods as samples

In [43]:
neighbourhood=census_sel.T.copy()
neighbourhood.columns.name=None
neighbourhood.head()

,Population Change 2011-2016,Population density per square kilometre,Male: 10 to 14 years,Female: 10 to 14 years,Male: 15 to 19 years,Female: 15 to 19 years,Male: 20 to 24 years,Female: 20 to 24 years,Male: 25 to 29 years,Female: 25 to 29 years,Male: 30 to 34 years,Female: 30 to 34 years,Male: 35 to 39 years,Female: 35 to 39 years,Male: 40 to 44 years,Female: 40 to 44 years,Male: 45 to 49 years,Female: 45 to 49 years,Male: 50 to 54 years,Female: 50 to 54 years,Male: 55 to 59 years,Female: 55 to 59 years,Male: 60 to 64 years,Female: 60 to 64 years,Male: 65 to 69 years,Female: 65 to 69 years,Male: 70 to 74 years,Female: 70 to 74 years,Male: 75 to 79 years,Female: 75 to 79 years,Male: 80 to 84 years,Female: 80 to 84 years,Male: 85 to 89 years,Female: 85 to 89 years,Male: 90 to 94 years,Female: 90 to 94 years,Male: 95 to 99 years,Female: 95 to 99 years,"No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Apprenticeship or trades certificate or diploma,"College, CEGEP or other non-university certificate or diploma",University certificate or diploma below bachelor level,Bachelor's degree,University certificate or diploma above bachelor level,"Degree in medicine, dentistry, veterinary medicine or optometry",Master's degree,Earned doctorate,Married or living common law,Never married,Separated,Divorced,Widowed,Single-detached house,Apartment in a building that has five or more storeys,Semi-detached house,Row house,Apartment or flat in a duplex,Apartment in a building that has fewer than five storeys,1 person,2 persons,3 persons,4 persons,5 or more persons,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999",Non-immigrants,Before 1981,1981 to 1990,1991 to 2000,2001 to 2010,2011 to 2016,Non-permanent residents,North American Aboriginal origins,Other North American origins,British Isles origins,French origins,Western European origins (except French origins),Northern European origins (except British Isles origins),Eastern European origins,Southern European origins,Jewish,Caribbean origins,Latin; Central and South American origins,African origins,West Central Asian and Middle Eastern origins,South Asian origins,East and Southeast Asian origins,Oceania origins
Agincourt North,-3.90%,"3,929",660,595,840,865,1015,975,1015,1005,835,935,680,775,760,910,890,1060,1160,1290,1060,1165,925,1070,925,985,590,690,490,575,380,485,210,350,100,160,25,60,3335,4240,520,2550,570,3090,215,105,735,85,14200,7355,570,1460,1690,3345,2120,805,1440,645,735,1350,2370,1995,1750,1645,150,105,165,450,340,565,450,510,455,475,930,755,695,640,500,935,465,510,"7,895","3,450","3,250","5,595","5,535","2,165",930,40,"1,345","1,500",265,350,80,265,"1,215",0,"1,445",470,535,475,"5,260","18,575",10
Agincourt South-Malvern West,8.00%,"3,034",460,475,780,690,1000,895,1045,975,820,835,625,715,610,745,760,885,970,975,850,915,710,795,630,690,425,450,350,405,240,350,155,205,45,100,20,20,1860,3410,460,2340,485,3270,215,120,920,85,11205,6605,460,1325,1080,2790,3145,330,515,695,610,1610,2325,1680,1335,1175,320,130,205,275,350,415,395,450,470,445,860,755,630,500,405,700,370,425,"7,190","2,830","2,255","3,805","4,460","1,735","1,175",105,"1,190","1,750",260,425,95,430,"1,635",20,"1,395",480,625,805,"4,170","13,075",0
Alderwood,1.30%,"2,435",225,255,285,290,355,310,355,350,410,430,455,450,420,445,440,430,515,515,540,485,390,400,300,325,205,210,155,180,105,210,65,130,35,70,10,5,795,1825,445,1735,195,1415,105,45,390,25,5735,3005,270,595,685,2840,255,545,85,330,560,1105,1440,885,795,390,70,35,80,85,160,175,170,210,185,225,390,355,365,310,265,605,445,470,"7,975","1,715",605,770,685,200,85,305,"2,355","3,940",855,"1,150",230,"2,415","3,34

### Clean the columns names

In [44]:
columns_formatted=[col.strip() for col in neighbourhood.columns]
print(len(columns_formatted))
neighbourhood.columns=columns_formatted

105


### Clean the data & convert to  float

In [45]:
for col in neighbourhood.columns:
    neighbourhood[col] = neighbourhood[col].str.replace('%', '')
    neighbourhood[col] = neighbourhood[col].str.replace(',', '')
    neighbourhood[col] = neighbourhood[col].astype(float)

In [46]:
neighbourhood.head()

,Population Change 2011-2016,Population density per square kilometre,Male: 10 to 14 years,Female: 10 to 14 years,Male: 15 to 19 years,Female: 15 to 19 years,Male: 20 to 24 years,Female: 20 to 24 years,Male: 25 to 29 years,Female: 25 to 29 years,Male: 30 to 34 years,Female: 30 to 34 years,Male: 35 to 39 years,Female: 35 to 39 years,Male: 40 to 44 years,Female: 40 to 44 years,Male: 45 to 49 years,Female: 45 to 49 years,Male: 50 to 54 years,Female: 50 to 54 years,Male: 55 to 59 years,Female: 55 to 59 years,Male: 60 to 64 years,Female: 60 to 64 years,Male: 65 to 69 years,Female: 65 to 69 years,Male: 70 to 74 years,Female: 70 to 74 years,Male: 75 to 79 years,Female: 75 to 79 years,Male: 80 to 84 years,Female: 80 to 84 years,Male: 85 to 89 years,Female: 85 to 89 years,Male: 90 to 94 years,Female: 90 to 94 years,Male: 95 to 99 years,Female: 95 to 99 years,"No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Apprenticeship or trades certificate or diploma,"College, CEGEP or other non-university certificate or diploma",University certificate or diploma below bachelor level,Bachelor's degree,University certificate or diploma above bachelor level,"Degree in medicine, dentistry, veterinary medicine or optometry",Master's degree,Earned doctorate,Married or living common law,Never married,Separated,Divorced,Widowed,Single-detached house,Apartment in a building that has five or more storeys,Semi-detached house,Row house,Apartment or flat in a duplex,Apartment in a building that has fewer than five storeys,1 person,2 persons,3 persons,4 persons,5 or more persons,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999",Non-immigrants,Before 1981,1981 to 1990,1991 to 2000,2001 to 2010,2011 to 2016,Non-permanent residents,North American Aboriginal origins,Other North American origins,British Isles origins,French origins,Western European origins (except French origins),Northern European origins (except British Isles origins),Eastern European origins,Southern European origins,Jewish,Caribbean origins,Latin; Central and South American origins,African origins,West Central Asian and Middle Eastern origins,South Asian origins,East and Southeast Asian origins,Oceania origins
Agincourt North,-3.9,3929.0,660.0,595.0,840.0,865.0,1015.0,975.0,1015.0,1005.0,835.0,935.0,680.0,775.0,760.0,910.0,890.0,1060.0,1160.0,1290.0,1060.0,1165.0,925.0,1070.0,925.0,985.0,590.0,690.0,490.0,575.0,380.0,485.0,210.0,350.0,100.0,160.0,25.0,60.0,3335.0,4240.0,520.0,2550.0,570.0,3090.0,215.0,105.0,735.0,85.0,14200.0,7355.0,570.0,1460.0,1690.0,3345.0,2120.0,805.0,1440.0,645.0,735.0,1350.0,2370.0,1995.0,1750.0,1645.0,150.0,105.0,165.0,450.0,340.0,565.0,450.0,510.0,455.0,475.0,930.0,755.0,695.0,640.0,500.0,935.0,465.0,510.0,7895.0,3450.0,3250.0,5595.0,5535.0,2165.0,930.0,40.0,1345.0,1500.0,265.0,350.0,80.0,265.0,1215.0,0.0,1445.0,470.0,535.0,475.0,5260.0,18575.0,10.0
Agincourt South-Malvern West,8.0,3034.0,460.0,475.0,780.0,690.0,1000.0,895.0,1045.0,975.0,820.0,835.0,625.0,715.0,610.0,745.0,760.0,885.0,970.0,975.0,850.0,915.0,710.0,795.0,630.0,690.0,425.0,450.0,350.0,405.0,240.0,350.0,155.0,205.0,45.0,100.0,20.0,20.0,1860.0,3410.0,460.0,2340.0,485.0,3270.0,215.0,120.0,920.0,85.0,11205.0,6605.0,460.0,1325.0,1080.0,2790.0,3145.0,330.0,515.0,695.0,610.0,1610.0,2325.0,1680.0,1335.0,1175.0,320.0,130.0,205.0,275.0,350.0,415.0,395.0,450.0,470.0,445.0,860.0,755.0,630.0,500.0,405.0,700.0,370.0,425.0,7190.0,2830.0,2255.0,3805.0,4460.0,1735.0,1175.0,105.0,1190.0,1750.0,260.0,425.0,95.0,430.0,1635.0,20.0,1395.0,480.0,625.0,805.0,4170.0,13075.0,0.0
Alderwood,1.3,2435.0,225.0,255.0,285.0,290.0,355.0,310.0,355.0,350.0,410.0,430.0,455.0,450.0

### Combine the age columns

In [47]:
cols_modify=[]
for col in neighbourhood.columns:
    if 'years' in col:
        cols_modify.append(col)

In [48]:
for col in cols_modify:
    print(col)

Male: 10 to 14 years
Female: 10 to 14 years
Male: 15 to 19 years
Female: 15 to 19 years
Male: 20 to 24 years
Female: 20 to 24 years
Male: 25 to 29 years
Female: 25 to 29 years
Male: 30 to 34 years
Female: 30 to 34 years
Male: 35 to 39 years
Female: 35 to 39 years
Male: 40 to 44 years
Female: 40 to 44 years
Male: 45 to 49 years
Female: 45 to 49 years
Male: 50 to 54 years
Female: 50 to 54 years
Male: 55 to 59 years
Female: 55 to 59 years
Male: 60 to 64 years
Female: 60 to 64 years
Male: 65 to 69 years
Female: 65 to 69 years
Male: 70 to 74 years
Female: 70 to 74 years
Male: 75 to 79 years
Female: 75 to 79 years
Male: 80 to 84 years
Female: 80 to 84 years
Male: 85 to 89 years
Female: 85 to 89 years
Male: 90 to 94 years
Female: 90 to 94 years
Male: 95 to 99 years
Female: 95 to 99 years


In [49]:
neighbourhood['Male: 15 to 24 years']=neighbourhood['Male: 15 to 19 years']+neighbourhood['Male: 20 to 24 years']
neighbourhood['Male: 25 to 34 years']=neighbourhood['Male: 25 to 29 years']+neighbourhood['Male: 30 to 34 years']
neighbourhood['Male: 35 to 44 years']=neighbourhood['Male: 35 to 39 years']+neighbourhood['Male: 40 to 44 years']
neighbourhood['Male: 45 to 54 years']=neighbourhood['Male: 45 to 49 years']+neighbourhood['Male: 50 to 54 years']
neighbourhood['Male: 55 to 64 years']=neighbourhood['Male: 55 to 59 years']+neighbourhood['Male: 60 to 64 years']
neighbourhood['Male: 65 and above']=neighbourhood['Male: 65 to 69 years']+neighbourhood['Male: 70 to 74 years']+neighbourhood['Male: 75 to 79 years']+neighbourhood['Male: 80 to 84 years']+neighbourhood['Male: 85 to 89 years']+neighbourhood['Male: 90 to 94 years']+neighbourhood['Male: 95 to 99 years']

In [50]:
neighbourhood['Female: 15 to 24 years']=neighbourhood['Female: 15 to 19 years']+neighbourhood['Female: 20 to 24 years']
neighbourhood['Female: 25 to 34 years']=neighbourhood['Female: 25 to 29 years']+neighbourhood['Female: 30 to 34 years']
neighbourhood['Female: 35 to 44 years']=neighbourhood['Female: 35 to 39 years']+neighbourhood['Female: 40 to 44 years']
neighbourhood['Female: 45 to 54 years']=neighbourhood['Female: 45 to 49 years']+neighbourhood['Female: 50 to 54 years']
neighbourhood['Female: 55 to 64 years']=neighbourhood['Female: 55 to 59 years']+neighbourhood['Female: 60 to 64 years']
neighbourhood['Female: 65 and above']=neighbourhood['Female: 65 to 69 years']+neighbourhood['Female: 70 to 74 years']+neighbourhood['Female: 75 to 79 years']+neighbourhood['Female: 80 to 84 years']+neighbourhood['Female: 85 to 89 years']+neighbourhood['Female: 90 to 94 years']+neighbourhood['Female: 95 to 99 years']

In [51]:
neighbourhood.head()

,Population Change 2011-2016,Population density per square kilometre,Male: 10 to 14 years,Female: 10 to 14 years,Male: 15 to 19 years,Female: 15 to 19 years,Male: 20 to 24 years,Female: 20 to 24 years,Male: 25 to 29 years,Female: 25 to 29 years,Male: 30 to 34 years,Female: 30 to 34 years,Male: 35 to 39 years,Female: 35 to 39 years,Male: 40 to 44 years,Female: 40 to 44 years,Male: 45 to 49 years,Female: 45 to 49 years,Male: 50 to 54 years,Female: 50 to 54 years,Male: 55 to 59 years,Female: 55 to 59 years,Male: 60 to 64 years,Female: 60 to 64 years,Male: 65 to 69 years,Female: 65 to 69 years,Male: 70 to 74 years,Female: 70 to 74 years,Male: 75 to 79 years,Female: 75 to 79 years,Male: 80 to 84 years,Female: 80 to 84 years,Male: 85 to 89 years,Female: 85 to 89 years,Male: 90 to 94 years,Female: 90 to 94 years,Male: 95 to 99 years,Female: 95 to 99 years,"No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Apprenticeship or trades certificate or diploma,"College, CEGEP or other non-university certificate or diploma",University certificate or diploma below bachelor level,Bachelor's degree,University certificate or diploma above bachelor level,"Degree in medicine, dentistry, veterinary medicine or optometry",Master's degree,Earned doctorate,Married or living common law,Never married,Separated,Divorced,Widowed,Single-detached house,Apartment in a building that has five or more storeys,Semi-detached house,Row house,Apartment or flat in a duplex,Apartment in a building that has fewer than five storeys,1 person,2 persons,3 persons,4 persons,5 or more persons,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999",Non-immigrants,Before 1981,1981 to 1990,1991 to 2000,2001 to 2010,2011 to 2016,Non-permanent residents,North American Aboriginal origins,Other North American origins,British Isles origins,French origins,Western European origins (except French origins),Northern European origins (except British Isles origins),Eastern European origins,Southern European origins,Jewish,Caribbean origins,Latin; Central and South American origins,African origins,West Central Asian and Middle Eastern origins,South Asian origins,East and Southeast Asian origins,Oceania origins,Male: 15 to 24 years,Male: 25 to 34 years,Male: 35 to 44 years,Male: 45 to 54 years,Male: 55 to 64 years,Male: 65 and above,Female: 15 to 24 years,Female: 25 to 34 years,Female: 35 to 44 years,Female: 45 to 54 years,Female: 55 to 64 years,Female: 65 and above
Agincourt North,-3.9,3929.0,660.0,595.0,840.0,865.0,1015.0,975.0,1015.0,1005.0,835.0,935.0,680.0,775.0,760.0,910.0,890.0,1060.0,1160.0,1290.0,1060.0,1165.0,925.0,1070.0,925.0,985.0,590.0,690.0,490.0,575.0,380.0,485.0,210.0,350.0,100.0,160.0,25.0,60.0,3335.0,4240.0,520.0,2550.0,570.0,3090.0,215.0,105.0,735.0,85.0,14200.0,7355.0,570.0,1460.0,1690.0,3345.0,2120.0,805.0,1440.0,645.0,735.0,1350.0,2370.0,1995.0,1750.0,1645.0,150.0,105.0,165.0,450.0,340.0,565.0,450.0,510.0,455.0,475.0,930.0,755.0,695.0,640.0,500.0,935.0,465.0,510.0,7895.0,3450.0,3250.0,5595.0,5535.0,2165.0,930.0,40.0,1345.0,1500.0,265.0,350.0,80.0,265.0,1215.0,0.0,1445.0,470.0,535.0,475.0,5260.0,18575.0,10.0,1855.0,1850.0,1440.0,2050.0,1985.0,2720.0,1840.0,1940.0,1685.0,2350.0,2235.0,3305.0
Agincourt South-Malvern West,8.0,3034.0,460.0,475.0,780.0,690.0,1000.0,895.0,1045.0,975.0,820.0,835.0,625.0,715.0,610.0,745.0,760.0,885.0,970.0,975.0,850.0,915.0,710.0,795.0,630.0,690.0,425.0,450.0,350.0,405.0,240.0,350.0,155.0,205.0,45.0,100.0,20.0,20.0,1860.0,3410.0,460.0,2340.0,485.0,3270.0,215.0,120.0,920.0,85.0,11205.0,6605.0,460.0,1325.0,1080.0,2790.0,3145.0,330.0,515.0,695.0,610.0,1610.0,2325.0,1680.0,1335.0,1175.0,320.

### drop the old columns

In [52]:
neighbourhood.drop(columns=cols_modify,inplace=True)

In [53]:
neighbourhood.shape

(140, 81)

### combine the salary columns

In [54]:
cols_modify_2=[]
for col in neighbourhood.columns:
    if '$' in col:
        cols_modify_2.append(col)
print(cols_modify_2)

['Under $5,000', '$5,000 to $9,999', '$10,000 to $14,999', '$15,000 to $19,999', '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999', '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999', '$50,000 to $59,999', '$60,000 to $69,999', '$70,000 to $79,999', '$80,000 to $89,999', '$90,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999', '$150,000 to $199,999']


In [55]:
neighbourhood['Under $10,000']=neighbourhood['Under $5,000']+neighbourhood['$5,000 to $9,999']
neighbourhood['$10,000 to $19,999']=neighbourhood['$10,000 to $14,999']+neighbourhood['$15,000 to $19,999']
neighbourhood['$20,000 to $29,999']=neighbourhood['$20,000 to $24,999']+neighbourhood['$25,000 to $29,999']
neighbourhood['$30,000 to $39,999']=neighbourhood['$30,000 to $34,999']+neighbourhood['$35,000 to $39,999']
neighbourhood['$40,000 to $49,999']=neighbourhood['$40,000 to $44,999']+neighbourhood['$45,000 to $49,999']

In [56]:
neighbourhood.drop(columns=cols_modify_2[0:10],inplace=True)

In [57]:
neighbourhood.shape

(140, 76)

In [58]:
neighbourhood.to_csv('/Users/Vineeth/Google Drive/Insight_project/neighbourhood-cleaned-2.csv',sep=',')